<a href="https://colab.research.google.com/github/lucasjsbarbosa/analise-de-sentimento-com-python-flask-e-api-da-open-ai/blob/main/analise_de_sentimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import openai
import os
import pandas as pd
from flask import Flask, request, render_template, send_file
from io import BytesIO
import xlsxwriter
import base64

In [ ]:
openai.api_key = os.environ["OPENAI_API_KEY"]
#Por motivos óbvios retirei minha KEY

## Função de Análise de Sentimento

In [ ]:
def analisa_sentimento(text):
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=f"This is a sentiment analysis task.\nPlease analyze the sentiment of the following text:\n{text}\nSentiment:",
        temperature=0.2,
        max_tokens=1,
        stop=["\n"]
    )

    sentiment = response.choices[0].text.strip()
    return sentiment

## Função de Processamento do Arquivo

In [ ]:
def processa_planilha(file):
    df = pd.read_excel(file)

    columns_to_drop = ['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 5', 'Unnamed: 7', 'Unnamed: 8']
    df = df.drop(columns=columns_to_drop, axis=1)

    df = df.dropna()

    df.rename(columns={'ExportComments.com': 'id', 'Unnamed: 6': 'review', 'Unnamed: 4': 'data'}, inplace=True)

    sentimentos = []
    for text in df['review'].values:
        sentimentos.append(analisa_sentimento(text))

    df['sentimento'] = sentimentos

    df['sentimento'] = df['sentimento'].apply(lambda x: "Neutral" if x == "" or x == "Unknown" else x)

    df['sentimento'] = df['sentimento'].str.title()

    return df

## Criação da API

In [ ]:
app = Flask(__name__, template_folder='template')

In [ ]:
@app.route('/', methods=['GET', 'POST'])

def upload_file():
    if request.method == 'POST':
        file = request.files['file']
        if file and file.filename.endswith('.xlsx'):

            # processa planilha
            result_df = processa_planilha(file)

            # cria um arquivo na memória
            excel_buffer = BytesIO()
            excel_writer = pd.ExcelWriter(excel_buffer, engine='xlsxwriter')
            result_df.to_excel(excel_writer, sheet_name='Sentiment Results', index=False)
            excel_writer.save()
            excel_buffer.seek(0)

            # converter o buffer para uma string em base64 para transformar o conteúdo binário
            # do arquivo Excel em uma representação de texto que pode ser incluída em uma URL ou em um documento HTML
            excel_base64 = base64.b64encode(excel_buffer.read()).decode('utf-8')

            return render_template('results.html', results=result_df.to_dict('records'), excel_base64=excel_base64)

    return render_template('upload.html')

@app.route('/download_excel', methods=['GET'])
def download_excel():
    excel_base64 = request.args.get('excel_base64')
    if excel_base64:
        excel_buffer = BytesIO(base64.b64decode(excel_base64.encode('utf-8')))
        df = pd.read_excel(excel_buffer)
        new_excel_buffer = BytesIO()
        df.to_excel(new_excel_buffer, index=False)
        new_excel_buffer.seek(0)
        response = send_file(
            new_excel_buffer,
            mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
        )

        # instruir o navegador a tratar o conteúdo como um anexo para download, em vez de tentar exibi-lo diretamente
        response.headers['Content-Disposition'] = 'attachment; filename=sentiment_results.xlsx'
        return response
    return "File not found", 404

In [ ]:
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Oct/2023 13:57:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2023 13:58:14] "POST / HTTP/1.1" 200 -
